In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from Config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kapaa,22.08,-159.32,82.4,74,20,11.41,US,2020-08-25 03:43:51
1,1,Vaini,-21.20,-175.20,78.8,78,20,8.05,TO,2020-08-25 03:44:48
2,2,Bluff,-46.60,168.33,45.0,71,62,15.99,NZ,2020-08-25 03:44:48
3,3,Mandan,46.83,-100.89,73.4,83,75,9.17,US,2020-08-25 03:44:48
4,4,Tuktoyaktuk,69.45,-133.04,53.6,87,20,8.05,CA,2020-08-25 03:44:48


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kapaa,22.08,-159.32,82.40,74,20,11.41,US,2020-08-25 03:43:51
1,1,Vaini,-21.20,-175.20,78.80,78,20,8.05,TO,2020-08-25 03:44:48
5,5,Hilo,19.73,-155.09,82.40,65,75,5.82,US,2020-08-25 03:44:48
10,10,Salalah,17.02,54.09,75.20,100,90,8.05,OM,2020-08-25 03:43:06
13,13,Butaritari,3.07,172.79,82.15,70,100,7.92,KI,2020-08-25 03:44:48
16,16,Kavaratti,10.57,72.64,82.40,74,20,9.17,IN,2020-08-25 03:44:49
20,20,Washington,47.50,-120.50,78.80,31,1,14.99,US,2020-08-25 03:39:52
21,21,Tautira,-17.73,-149.15,82.40,65,20,9.17,PF,2020-08-25 03:44:49
22,22,Boyolangu,-8.12,111.89,84.67,60,27,9.57,ID,2020-08-25 03:44:49
23,23,Mitchell,43.71,-98.03,82.40,65,1,11.41,US,2020-08-25 03:44:49


In [13]:
preferred_cities_df.count()

City_ID       186
City          186
Lat           186
Lng           186
Max Temp      186
Humidity      186
Cloudiness    186
Wind Speed    186
Country       186
Date          186
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,82.40,22.08,-159.32,
1,Vaini,TO,78.80,-21.20,-175.20,
5,Hilo,US,82.40,19.73,-155.09,
10,Salalah,OM,75.20,17.02,54.09,
13,Butaritari,KI,82.15,3.07,172.79,
16,Kavaratti,IN,82.40,10.57,72.64,
20,Washington,US,78.80,47.50,-120.50,
21,Tautira,PF,82.40,-17.73,-149.15,
22,Boyolangu,ID,84.67,-8.12,111.89,
23,Mitchell,US,82.40,43.71,-98.03,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [23]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,82.40,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
1,Vaini,TO,78.80,-21.20,-175.20,Keleti Beach Resort
5,Hilo,US,82.40,19.73,-155.09,Hilo Hawaiian Hotel
10,Salalah,OM,75.20,17.02,54.09,HAMDAN PLAZA HOTEL
13,Butaritari,KI,82.15,3.07,172.79,Isles Sunset Lodge
16,Kavaratti,IN,82.40,10.57,72.64,Bismillah Hotel And Lodging
20,Washington,US,78.80,47.50,-120.50,"Cashmere Cider Mill Guest Suites, Weddings & T..."
21,Tautira,PF,82.40,-17.73,-149.15,Pension Oaoa
22,Boyolangu,ID,84.67,-8.12,111.89,Yos ega cv
23,Mitchell,US,82.40,43.71,-98.03,Kelly Inn & Suites


In [27]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [31]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [32]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))